In [1]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
disease_labels = [
    "Atelectasis", "Cardiomegaly", "Consolidation", "Edema", "Effusion",
    "Emphysema", "Fibrosis", "Hernia", "Infiltration", "Mass", 
    "No Finding", "Nodule", "Pleural_Thickening", "Pneumonia", "Pneumothorax"
]

In [3]:
csv_file = '/kaggle/input/zsl-dataset1/Data_Entry_2017.csv'  # Path to the CSV file
image_dir = '/kaggle/input/zsl-dataset4'  

In [4]:
df = pd.read_csv(csv_file)
available_images = {img for img in os.listdir(image_dir) if img.endswith('.png') or img.endswith('.jpg')}
df = df[df['Image Index'].isin(available_images)].reset_index(drop=True)


In [5]:
class MedicalImageDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.label_to_index = {label: idx for idx, label in enumerate(disease_labels)}

    def __len__(self):
        return len(self.dataframe)

    def encode_labels(self, labels):
        label_vector = torch.zeros(len(disease_labels), dtype=torch.float32)
        for label in labels.split('|'):
            if label in self.label_to_index:
                label_vector[self.label_to_index[label]] = 1
        return label_vector

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('L')  # Convert to grayscale if required

        # Apply transformations to the image
        if self.transform:
            image = self.transform(image)

        # Encode multilabel for the "Finding Labels" column
        labels = self.dataframe.iloc[idx, 1]
        label_vector = self.encode_labels(labels)

        return {'image': image, 'label': label_vector}

In [6]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Instantiate the dataset and dataloader
dataset = MedicalImageDataset(dataframe=df, img_dir=image_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [7]:
class SimpleCNN(nn.Module):
    def __init__(self, num_labels):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, num_labels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

In [8]:
model = SimpleCNN(num_labels=len(disease_labels))
criterion = nn.BCELoss()  # Binary Cross-Entropy for multilabel classification
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

SimpleCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=65536, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=15, bias=True)
  (sigmoid): Sigmoid()
)

In [10]:
import torch.nn.functional as F
num_epochs = 5  # Set number of epochs
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in dataloader:
        images = batch['image'].to(device)
        labels = batch['label'].to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print average loss per epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(dataloader):.4f}")

print("Training complete.")

Epoch [1/5], Loss: 0.2752
Epoch [2/5], Loss: 0.2491
Epoch [3/5], Loss: 0.2366
Epoch [4/5], Loss: 0.2258
Epoch [5/5], Loss: 0.2143
Training complete.
